In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install graphviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 39.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.1 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=26fbdf518a0934cf978adb0987aaf5fcad5cdb39810fcd10ab950fd4cdca904b
  Stored in directory: /root/.cache/pip/wheels/2d/f0/a8/1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully built validators
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

In [ ]:
%%writefile sim.py
import streamlit as st
# import json
import graphviz as gv
from google.colab import files
import json
import sys


def resolve_dead(delta, states, DFA):
    check = {}
    for state in states:
        check.update({state: False})
    for state in DFA['start_states']:
        check[state] = True
    for edge in delta:
        check[edge[2]] = True
    return [edge for edge in delta if check[edge[0]]], [state for state in check if check[state]]


def visit(state, check, delta):
    if check[state]:
        return
    check[state] = True
    for s in [edge[2] for edge in delta if edge[0] == state]:
        visit(s, check, delta)


def resolve_inaccessible(delta, states, DFA):
    check = {}
    for state in states:
        check.update({state: False})
    for state in DFA['start_states']:
        visit(state, check, delta)
    return [edge for edge in delta if check[edge[0]]], [state for state in check if check[state]]


def equivalent(state1, state2, arr, DFA, alpha_delta):
    check = {l: False for l in DFA['letters']}
    for l in alpha_delta:
        newlist = [edge for edge in alpha_delta[l]
                   if (edge[0] in [state1, state2])]
        if not newlist:
            check[l] = True
        elif len(newlist) == 1:
            return False
        else:
            final1, final2 = newlist[0][2], newlist[1][2]
            for group in arr:
                if final1 in group and final2 in group:
                    check[l] = True
                    break
                elif final1 in group or final2 in group:
                    return False
    if set(check.values()) == {True}:
        return True
    return False


def k_equivalence(delta, k, hist, DFA, valid_states, alpha_delta):
    partition = []
    if k == 0:
        partition.append(DFA['final_states'])
        partition.append(
            [s for s in valid_states if s not in DFA['final_states']])
        hist.append(partition)

        return False
    else:
        old_partition = hist[-1]
        for group in old_partition:
            for state in group:
                if state in [s for grp in partition for s in grp]:
                    continue
                newgrp = [state]
                for s in group:
                    if s == state:
                        continue
                    if equivalent(state, s, old_partition, DFA, alpha_delta):
                        newgrp.append(s)
                partition.append(newgrp)
        hist.append(partition)

        if sorted(hist[-1]) == sorted(hist[-2]):
            return True
        else:
            return False

def mindfa(DFA):
    # with open(inputfile, 'r') as f:
    #     DFA = json.load(f)

    delta = DFA['transition_function'].copy()
    delta, valid_states = resolve_dead(delta, DFA['states'], DFA)
    delta, valid_states = resolve_inaccessible(delta, valid_states, DFA)
    alpha_delta = {l: [edge for edge in delta if edge[1] == l]
                   for l in DFA['letters']}

    k = 0
    hist = []
    flag = False
    while not flag:
        flag = k_equivalence(delta, k, hist, DFA, valid_states, alpha_delta)
        k += 1

    new_states = hist[-1]

    start_states = []
    for state in new_states:
        for s in DFA['start_states']:
            if s in state:
                start_states.append(state)
                break

    final_states = []
    for state in new_states:
        for s in DFA['final_states']:
            if s in state:
                final_states.append(state)
                break

    new_delta = []
    for state in new_states:
        for l in DFA['letters']:
            endstate = [edge[2]
                        for edge in alpha_delta[l] if edge[0] == state[0]]
            if not endstate:
                continue
            for s in new_states:
                if endstate[0] in s:
                    accept = s
                    break
            new_delta.append([state, l, accept])

    minDFA = {
        'states': new_states,
        'letters': DFA['letters'],
        'transition_function': new_delta,
        'start_states': start_states,
        'final_states': final_states,
    }

    output = json.dumps(minDFA, indent=4)
    with open('output.json', 'w') as f:
        f.write(output)


def visualize_finite_automata(data):
    states = data['states']
    letters = data['letters']
    transitions = data['transition_function']
    start_states = data['start_states']
    final_states = data['final_states']

    # Convert final state 2D list to comma-separated string
    for i, state in enumerate(final_states):
        if isinstance(state, list):
            final_states[i] = ",".join(state)
            
    # Rename 2D array of states as 1 state
    for i, state in enumerate(states):
        if isinstance(state, list):
            states[i] = ",".join(state)

    # Rename 2D array of states as 1 state in transition_function
    for i, transition in enumerate(transitions):
        if isinstance(transition[0], list):
            transitions[i][0] = ",".join(transition[0])
        if isinstance(transition[2], list):
            transitions[i][2] = ",".join(transition[2])

    # Initialize the graph
    graph = gv.Digraph(format='svg')
    graph.attr(rankdir='LR')

    # Add nodes
    for state in states:
        if state in final_states:
            graph.node(state, shape='doublecircle')
        else:
            graph.node(state)

    # Add edges
    for transition in transitions:
        start_state, letter, end_state = transition
        if letter == "$":
            graph.edge(start_state, end_state, label="ε")
        else:
            graph.edge(start_state, end_state, label=letter)

    # Return the graph object
    return graph

# Set the page width to a high value
st.set_page_config(page_title="Finite Automata Visualizer", layout="wide", initial_sidebar_state="collapsed")

# Streamlit app
st.title("DFA Minimizer")

# Upload JSON files
# file1 = st.file_uploader("Upload JSON file for automata 1", type=["json"])
with st.sidebar:
    st.header("Input")
    file1 = st.file_uploader("Upload JSON file for Given DFA", type=["json"])
# file2 = st.file_uploader("Upload JSON file for automata 2", type=["json"])
# with st.sidebar:
#     st.header("Output")
#     file2 = st.file_uploader("Upload JSON file for automata 2", type=["json"])

if file1 is not None:
    # Load JSON data
    data1 = json.load(file1)
    mindfa(data1)
    with open('output.json') as file2:
      data2 = json.load(file2)

    # Display the finite automata diagrams side by side
    col1, col2 = st.columns(2)

    # Create the graphs
    graph1 = visualize_finite_automata(data1)
    graph2 = visualize_finite_automata(data2)

    # Display the graphs in the columns
    with col1:
        st.header("Input (DFA)")
        st.graphviz_chart(graph1, use_container_width=True)
    with col2:
        st.header("Output (Minimized DFA)")
        st.graphviz_chart(graph2, use_container_width=True)
# st.markdown("---")
# st.write("Created by Het Patel")


Writing sim.py


In [ ]:
!ls

sample_data  sim.py


In [ ]:
!ngrok authtoken XXXXX

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!ngrok

NAME:
  ngrok - tunnel local ports to public URLs and inspect traffic

USAGE:
  ngrok [command] [flags]

DESCRIPTION: 
  ngrok exposes local networked services behinds NATs and firewalls to the
  public internet over a secure tunnel. Share local websites, build/test
  webhook consumers and self-host personal services.
  Detailed help for each command is available with 'ngrok help <command>'.
  Open http://localhost:4040 for ngrok's web interface to inspect traffic.

Author:
  ngrok - <support@ngrok.com>

TERMS OF SERVICE: https://ngrok.com/tos

EXAMPLES: 
  ngrok http 80                           # secure public URL for port 80 web server
  ngrok http --domain baz.ngrok.dev 8080  # port 8080 available at baz.ngrok.dev
  ngrok http foo.dev:80                   # tunnel to host:port instead of localhost
  ngrok http https://localhost            # expose a local https server
  ngrok tcp 22                            # tunnel arbitrary TCP traffic to port 22
  ngrok tls --domain=foo.com 44

In [ ]:
from pyngrok import ngrok

In [ ]:
!pgrep streamlit

In [ ]:
!streamlit run sim.py & npx localtunnel --port 8501

[##................] | fetchMetadata: sill resolveWithNewModule y18n@5.0.8 chec


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.236.230.88:8501

npx: installed 22 in 4.159s
your url is: https://cute-clocks-flash-35-236-230-88.loca.lt
